<a href="https://colab.research.google.com/github/abhishek-m928/Designing-a-Multi-Level-PUF-for-Robust-Hardware-Security/blob/main/submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import sklearn
from scipy.linalg import khatri_rao
from itertools import combinations
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import time as tm

In [ ]:
c = np.loadtxt('public_trn.txt')
z = c[:, :-1]  # All columns except the last
y_train = c[:, -1]

FileNotFoundError: public_trn.txt not found.

In [ ]:
def my_map(z):
    # Step 1: Convert z (binary) to d ∈ {-1, 1}
    d = 1 - 2 * z  # vector of shape (8,)

    # Step 2: Build phi vector
    phi = list(d)  # original d1 to d8
    temp = d[0]
    for j in range(1, len(d)):
        temp *= d[j]
        phi.append(temp)

    phi = np.array(phi)  # shape (15,)
    n_features = len(phi)
    # feature_map = np.zeros((1, (n_features * (n_features - 1)) // 2))  # 105 pairs (for phi size 15)

    # k = 0  # To index into the feature_map
    # for i in range(n_features):
    #     for j in range(i + 1, n_features):
    #         feature_map[0, k] = phi[i] * phi[j]
    #         k += 1  # Increment index in feature_map
    phi_matrix = np.reshape(phi, (len(phi), 1))
    feat = khatri_rao(phi_matrix,phi_matrix)
    return feat  # shape (1, 105) j]

In [ ]:
def my_fit(X, y):
    # Assuming X has shape (n_samples, 8) and y has shape (n_samples,)

    # Step 1: Apply my_map to each sample to get the feature map
    features = np.array([my_map(x) for x in X])  # shape (n_samples, 1, 105)
    features = features.reshape(X.shape[0], -1)  # Flatten to shape (n_samples, 105)

    # Step 2: Train SVM classifier
    # model = SVC(kernel='poly',degree=8)
    model = LinearSVC(loss='hinge',C=1)
    # model = LogisticRegression(C= 1, penalty='l1',solver='liblinear')
    model.fit(features, y)
    print(model.intercept_,model.coef_)
    return model



In [ ]:
ti = tm.perf_counter()
svm = my_fit(z,y_train)
tc =  tm.perf_counter()
print(tc-ti)

In [ ]:
import numpy as np
import time as tm
from sklearn.metrics import accuracy_score

# Load the test data
ct = np.loadtxt('public_tst.txt')

# Split into features and labels
z = ct[:, :-1]
y_test = ct[:, -1]

# Number of runs
runs = 8
times = []

for _ in range(runs):
    l = []
    ti = tm.perf_counter()
    for i in range(z.shape[0]):
        row = my_map(z[i])  # Assuming this returns shape (1, 105)
        l.append(row.flatten())
    tc = tm.perf_counter()
    times.append((tc - ti) / z.shape[0])  # Time per sample

# Convert to array once after timing
l = np.array(l)

# Predict using trained SVM
yp = svm.predict(l)

# Calculate accuracy
acc = accuracy_score(yp, y_test)

print(f"Accuracy: {acc * 100:.2f}%")
print(f"Average mapping time per sample over {runs} runs: {np.mean(times):.6f} seconds")
print(f"Feature map shape for one sample: {row.shape}")


In [ ]:
def my_decode(X):
    p = np.zeros(64)
    q = np.zeros(64)
    r = np.zeros(64)
    s = np.zeros(64)

    for i in range(63):
        if X[i] >= 0:
            p[i] = r[i] = X[i]
        else:
            q[i] = s[i] = -X[i]

    total = X[63] + X[64]
    difference = X[63] - X[64]

    if total >= 0:
        p[63] = total
    else:
        q[63] = -total

    if difference >= 0:
        r[63] = difference
    else:
        s[63] = -difference

    return p, q, r, s

In [ ]:
# def my_decode(model_params):
#     """Recovers non-negative delays from linear model parameters"""
#     if model_params is None or len(model_params) != 65:
#         raise ValueError("model_params must be a 65-element array")

#     w = model_params[:-1]
#     b = model_params[-1]
#     n = len(w)

#     # Backward calculation of beta and alpha
#     alpha = np.zeros(n)
#     beta = np.zeros(n)
#     beta[n-1] = b
#     for i in range(0, n-1):
#         alpha[i] = w[i]
#     alpha[n-1] = w[n-1] - beta[n-1]

#     p = np.zeros(n)
#     q = np.zeros(n)
#     r = np.zeros(n)
#     s = np.zeros(n)
#     pq = alpha + beta
#     rs = alpha - beta
#     # Non-negative delays
#     for i in range(n):
#         if pq[i] >= 0:
#             p[i] = pq[i]
#         else:
#             q[i] = -pq[i]
#     for i in range(n):
#         if rs[i] >= 0:
#             r[i] = rs[i]
#         else:
#             s[i] = -rs[i]

#     return p, q, r, s

In [ ]:

def invert_model(w):
    alpha = w[:-1]
    beta = w[1:] - w[:-1]
    d = alpha + beta
    c = alpha - beta

    # Decompose into non-negative components
    p = np.maximum(d, 0)
    q = np.maximum(-d, 0)
    r = np.maximum(c, 0)
    s = np.maximum(-c, 0)

    return p, q, r, s


In [ ]:
def get_model( p, q, r, s ):
  p = np.maximum( p, 0 )
  q = np.maximum( q, 0 )
  r = np.maximum( r, 0 )
  s = np.maximum( s, 0 )
  d = p - q
  c = r - s
  alpha = ( d + c ) / 2
  beta = ( d - c ) / 2
  w = np.zeros( ( len( alpha ) + 1, )  )
  w[:-1] += alpha
  w[1:] += beta
  return w

In [ ]:
import numpy as np
import time as tm
W = np.loadtxt( "public_mod.txt" )
( n_models, dims ) = W.shape
t_decode = 0
m_dist = 0
for t in range(5):
  for itr in range( n_models ):
    w = W[ itr, : ]
    tic = tm.perf_counter()
    p_hat, q_hat, r_hat, s_hat = invert_model( w )
    for qt in s_hat :
      if(qt<0):
        print(qt)
    toc = tm.perf_counter()
    t_decode += toc - tic
    w_hat = get_model( p_hat, q_hat, r_hat, s_hat )
    m_dist += np.linalg.norm( w - w_hat )

In [ ]:
print(m_dist/n_models)
print(t_decode/n_models)

In [ ]:
print("hello")

In [ ]:
!pip install PyMuPDF -qq
import fitz # PyMuPDF

doc = fitz.open("/content/CS771_Assignment_1___The_Optimisers.pdf")
content = ""
for page_num in range(doc.page_count):
    page = doc.load_page(page_num)
    content += page.get_text()

print(content[:1000])

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 82.2 MB/s eta 0:00:00
Assignment 1
The Optimisers
Group Members:
1.
Jonna Likith Sai
220483
2.
Sumit Rojaria
221104
3.
Abhishek Kumar
220046
4.
Vivek Goyal
221210
Date of Submission: May 3rd, 2025
University: Indian Institute of Technology Kanpur
Course: CS771 Introduction to Machine Learning
Instructor: Purushottam Kar
Total Marks: 150
Contents
1
Question 1: Multi-level PUF
3
1.1
Introduction and Notations . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
3
1.2
Mathematical derivation on how a single linear model can predict the responses . .
4
1.2.1
Proof for Linearity of an Arbiter PUF . . . . . . . . . . . . . . . . . . . .
4
1.2.2
Linearity of a Multi Level PUF . . . . . . . . . . . . . . . . . . . . . . . .
5
1.3
Dimensionality of the Feature Map . . . . . . . . . . . . . . . . . . . . . . . . . .
7
1.4
Best Kernel SVM with Hyper parameters
. . . . . . . . . . . . . . . . . . . . . .
8
2
Question 2 : Delay retriva